In [1]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import ipywidgets as widgets
from IPython.display import display, clear_output
import os
import json
import io

print("🚀 Starting Fragile Watermarking System...")

🚀 Starting Fragile Watermarking System...


In [2]:
# 1. Core Functions (Same as before)
def load_and_preprocess_image(image_path):
    """Load and preprocess image for watermarking"""
    try:
        img = Image.open(image_path)
        img_array = np.array(img)
        print(f"✅ Image loaded: {img_array.shape}")
        return img_array, 'color' if len(img_array.shape) == 3 else 'grayscale'
    except Exception as e:
        print(f"❌ Error loading image: {e}")
        return None, None

def text_to_binary(text):
    """Convert text to binary"""
    return ''.join(format(ord(char), '08b') for char in text)

def binary_to_text(binary_str):
    """Convert binary back to text"""
    text = ''
    for i in range(0, len(binary_str), 8):
        byte = binary_str[i:i+8]
        if len(byte) == 8:
            text += chr(int(byte, 2))
    return text

def embed_watermark_lsb(image_array, watermark_text, mode='color'):
    """Embed watermark using LSB"""
    binary_watermark = text_to_binary(watermark_text) + '00000000'
    watermark_length = len(binary_watermark)
    
    watermarked_image = image_array.copy()
    
    if mode == 'color':
        flat_pixels = watermarked_image.reshape(-1)
    else:
        flat_pixels = watermarked_image.reshape(-1)
    
    if watermark_length > len(flat_pixels):
        raise ValueError("❌ Watermark too large for image!")
    
    # Embed the watermark
    for i in range(watermark_length):
        flat_pixels[i] = (flat_pixels[i] & 0xFE) | int(binary_watermark[i])
    
    if mode == 'color':
        watermarked_image = flat_pixels.reshape(image_array.shape)
    else:
        watermarked_image = flat_pixels.reshape(image_array.shape)
    
    return watermarked_image, watermark_length

def extract_watermark_lsb(image_array, watermark_length, mode='color'):
    """Extract watermark from LSB"""
    if mode == 'color':
        flat_pixels = image_array.reshape(-1)
    else:
        flat_pixels = image_array.reshape(-1)
    
    binary_watermark = ''.join(str(flat_pixels[i] & 1) for i in range(watermark_length))
    return binary_to_text(binary_watermark)

def calculate_quality_metrics(original, watermarked):
    """Calculate image quality metrics"""
    mse = np.mean((original.astype(float) - watermarked.astype(float)) ** 2)
    psnr = 20 * np.log10(255.0 / np.sqrt(mse)) if mse > 0 else float('inf')
    return {'MSE': mse, 'PSNR': psnr}

def save_watermark_info(watermark_text, watermark_length, output_path):
    """Save watermark information to a JSON file for verification"""
    info = {
        'watermark_text': watermark_text,
        'watermark_length': watermark_length,
        'timestamp': np.datetime64('now').astype(str)
    }
    
    info_path = output_path.replace('.png', '_watermark_info.json')
    with open(info_path, 'w') as f:
        json.dump(info, f, indent=2)
    
    return info_path

def load_watermark_info(info_path):
    """Load watermark information from JSON file"""
    try:
        with open(info_path, 'r') as f:
            return json.load(f)
    except:
        return None

In [3]:
# 2. UNIVERSAL UPLOAD HANDLER - FIXED VERSION
def handle_uploaded_file(upload_widget, file_type='image'):
    """
    Universal upload handler that works with all ipywidgets versions
    Returns: file content and filename
    """
    if not upload_widget.value:
        return None, None
    
    try:
        upload_data = upload_widget.value
        
        # Debug info
        print(f"📦 Upload data type: {type(upload_data)}")
        
        # Method 1: New ipywidgets format (dict)
        if isinstance(upload_data, dict) and len(upload_data) > 0:
            first_key = list(upload_data.keys())[0]
            file_info = upload_data[first_key]
            content = file_info['content']
            filename = file_info['name']
            print(f"✅ Using Method 1 (dict) - File: {filename}")
            return content, filename
        
        # Method 2: Old ipywidgets format (list/tuple)
        elif isinstance(upload_data, (list, tuple)) and len(upload_data) > 0:
            file_info = upload_data[0]
            content = file_info['content']
            filename = file_info['name']
            print(f"✅ Using Method 2 (list) - File: {filename}")
            return content, filename
        
        # Method 3: Direct content access (fallback)
        else:
            print("❓ Unknown upload format, trying fallback...")
            # Try to extract content directly
            if hasattr(upload_data, 'content'):
                content = upload_data.content
                filename = f"uploaded_{file_type}.{'png' if file_type == 'image' else 'json'}"
                print(f"✅ Using Method 3 (fallback) - File: {filename}")
                return content, filename
        
    except Exception as e:
        print(f"❌ Upload handling error: {e}")
        return None, None
    
    return None, None

In [4]:
#3
def create_embedding_interface():
    """Interface for embedding watermarks"""
    
    upload = widgets.FileUpload(
        description='📁 Upload Image',
        accept='.jpg,.jpeg,.png,.bmp,.tiff',
        multiple=False,
        style={'description_width': 'initial'}
    )
    
    watermark_text = widgets.Textarea(
        value='© Copyright 2024',
        description='💧 Watermark Text:',
        layout=widgets.Layout(width='500px', height='80px'),
        style={'description_width': 'initial'}
    )
    
    embed_btn = widgets.Button(
        description='🔒 EMBED WATERMARK',
        button_style='success',
        layout=widgets.Layout(width='200px', height='40px')
    )
    
    output_area = widgets.Output()
    
    def embed_watermark(b):
        with output_area:
            clear_output()
            print("⏳ Processing your request...")
            
            # Get uploaded file using universal handler
            content, filename = handle_uploaded_file(upload, 'image')
            if content is None:
                print("❌ Please upload an image first!")
                return
            
            if not watermark_text.value.strip():
                print("❌ Please enter watermark text!")
                return
                
            try:
                # Create unique filename
                temp_path = f"temp_upload_{np.random.randint(1000, 9999)}.png"
                
                # Save uploaded image
                with open(temp_path, 'wb') as f:
                    f.write(content)
                
                print(f"✅ Saved uploaded image as: {temp_path}")
                
                # Load and process image
                original_image, image_mode = load_and_preprocess_image(temp_path)
                if original_image is None:
                    return
                
                print(f"🎨 Image mode: {image_mode}")
                print(f"📏 Image dimensions: {original_image.shape}")
                
                # Embed watermark
                print("💧 Embedding watermark...")
                watermarked, wm_length = embed_watermark_lsb(
                    original_image, watermark_text.value, image_mode
                )
                
                # Display comparison
                fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))
                
                if image_mode == 'color':
                    ax1.imshow(original_image)
                    ax2.imshow(watermarked)
                else:
                    ax1.imshow(original_image, cmap='gray')
                    ax2.imshow(watermarked, cmap='gray')
                
                ax1.set_title('📷 ORIGINAL IMAGE')
                ax2.set_title('🔒 WATERMARKED IMAGE')
                ax1.axis('off')
                ax2.axis('off')
                plt.tight_layout()
                plt.show()
                
                # Calculate quality metrics
                metrics_result = calculate_quality_metrics(original_image, watermarked)
                print("\n📊 QUALITY METRICS:")
                print(f"   PSNR: {metrics_result['PSNR']:.2f} dB (Higher = Better)")
                print(f"   MSE:  {metrics_result['MSE']:.2f} (Lower = Better)")
                
                # Save watermarked image
                output_path = 'my_watermarked_image.png'
                result_img = Image.fromarray(watermarked.astype(np.uint8))
                if image_mode == 'grayscale':
                    result_img = result_img.convert('L')
                result_img.save(output_path)
                
                # Save watermark info
                info_path = save_watermark_info(watermark_text.value, wm_length, output_path)
                
                print(f"\n💾 FILES SAVED:")
                print(f"   🔒 Watermarked Image: {output_path}")
                print(f"   📄 Watermark Info: {info_path}")
                
                print(f"\n✅ EMBEDDING COMPLETED SUCCESSFULLY!")
                print("   You can now switch to the verification tab to test your watermark.")
                
                # Show quick verification
                extracted = extract_watermark_lsb(watermarked, wm_length, image_mode)
                print(f"\n🔍 QUICK SELF-TEST:")
                print(f"   Extracted: {extracted}")
                print(f"   Status: {'✅ VALID' if watermark_text.value in extracted else '❌ INVALID'}")
                
            except Exception as e:
                print(f"❌ Error during embedding: {str(e)}")
                import traceback
                print(f"🔍 Detailed error: {traceback.format_exc()}")
            finally:
                # Cleanup
                if os.path.exists(temp_path):
                    os.remove(temp_path)
                    print(f"🧹 Cleaned up temporary file: {temp_path}")
    
    embed_btn.on_click(embed_watermark)
    
    return widgets.VBox([
        widgets.HTML("<h2>🔒 PHASE 1: EMBED WATERMARK</h2>"),
        widgets.HTML("<p>Upload an image and embed your copyright watermark</p>"),
        upload,
        watermark_text,
        embed_btn,
        output_area
    ])

In [5]:
# 4. Verification Interface
def create_verification_interface():
    """Interface for verifying watermarks"""
    
    image_upload = widgets.FileUpload(
        description='📁 Upload Image to Verify',
        accept='.jpg,.jpeg,.png,.bmp,.tiff',
        multiple=False
    )
    
    info_upload = widgets.FileUpload(
        description='📁 Upload Watermark Info (Optional)',
        accept='.json',
        multiple=False
    )
    
    manual_text = widgets.Textarea(
        value='',
        description='🔍 Expected Watermark:',
        layout=widgets.Layout(width='500px', height='60px'),
        placeholder='Enter the watermark text you expect to find...'
    )
    
    verify_btn = widgets.Button(
        description='🔍 VERIFY WATERMARK',
        button_style='info',
        layout=widgets.Layout(width='200px', height='40px')
    )
    
    output_area = widgets.Output()
    
    def verify_watermark(b):
        with output_area:
            clear_output()
            print("⏳ Starting verification...")
            
            # Get uploaded image
            image_content, image_filename = handle_uploaded_file(image_upload, 'image')
            if image_content is None:
                print("❌ Please upload an image to verify!")
                return
            
            try:
                # Save uploaded image
                temp_image_path = f"verify_temp_{np.random.randint(1000, 9999)}.png"
                with open(temp_image_path, 'wb') as f:
                    f.write(image_content)
                
                print(f"✅ Loaded image: {image_filename}")
                
                # Load image for verification
                verify_image, image_mode = load_and_preprocess_image(temp_image_path)
                if verify_image is None:
                    return
                
                # Determine what to verify against
                expected_text = manual_text.value.strip()
                watermark_length = None
                
                # Try to load from JSON info file
                info_content, info_filename = handle_uploaded_file(info_upload, 'json')
                if info_content:
                    temp_info_path = f"info_temp_{np.random.randint(1000, 9999)}.json"
                    with open(temp_info_path, 'wb') as f:
                        f.write(info_content)
                    
                    watermark_info = load_watermark_info(temp_info_path)
                    if watermark_info:
                        expected_text = watermark_info['watermark_text']
                        watermark_length = watermark_info['watermark_length']
                        print(f"✅ Loaded watermark info from: {info_filename}")
                
                if not expected_text:
                    print("❌ No watermark text provided! Use JSON file or enter text manually.")
                    return
                
                # Calculate watermark length if not provided
                if watermark_length is None:
                    watermark_length = len(text_to_binary(expected_text)) + 8
                    print("ℹ️  Using calculated watermark length")
                
                print(f"🔍 Verifying against: '{expected_text}'")
                print(f"📏 Watermark length: {watermark_length} bits")
                
                # Extract and verify
                extracted = extract_watermark_lsb(verify_image, watermark_length, image_mode)
                extracted_clean = extracted.replace('\x00', '').strip()
                
                print(f"\n📊 VERIFICATION RESULTS:")
                print(f"   Expected: '{expected_text}'")
                print(f"   Extracted: '{extracted_clean}'")
                
                # Calculate match percentage
                if expected_text in extracted_clean:
                    match_percent = 100.0
                    print(f"   ✅ PERFECT MATCH: 100%")
                    print(f"   🛡️  IMAGE IS AUTHENTIC")
                else:
                    # Calculate similarity
                    similarity = 0
                    min_len = min(len(expected_text), len(extracted_clean))
                    if min_len > 0:
                        for i in range(min_len):
                            if expected_text[i] == extracted_clean[i]:
                                similarity += 1
                        match_percent = (similarity / len(expected_text)) * 100
                    else:
                        match_percent = 0
                    
                    print(f"   ❌ MISMATCH: {match_percent:.1f}% similarity")
                    print(f"   ⚠️  IMAGE MAY HAVE BEEN TAMPERED WITH")
                
                # Display the verified image
                plt.figure(figsize=(10, 8))
                if image_mode == 'color':
                    plt.imshow(verify_image)
                else:
                    plt.imshow(verify_image, cmap='gray')
                
                if match_percent > 90:
                    plt.title(f'✅ AUTHENTIC IMAGE ({match_percent:.1f}% Match)', fontsize=14, color='green')
                else:
                    plt.title(f'❌ POSSIBLY TAMPERED ({match_percent:.1f}% Match)', fontsize=14, color='red')
                
                plt.axis('off')
                plt.tight_layout()
                plt.show()
                
            except Exception as e:
                print(f"❌ Verification error: {str(e)}")
                import traceback
                print(f"🔍 Detailed error: {traceback.format_exc()}")
            finally:
                # Cleanup
                for temp_file in [temp_image_path, 'watermark_info.json']:
                    if os.path.exists(temp_file):
                        os.remove(temp_file)
    
    verify_btn.on_click(verify_watermark)
    
    return widgets.VBox([
        widgets.HTML("<h2>🔍 PHASE 2: VERIFY WATERMARK</h2>"),
        widgets.HTML("<p>Upload a watermarked image to check its authenticity</p>"),
        image_upload,
        info_upload,
        manual_text,
        verify_btn,
        output_area
    ])

In [6]:
# 5. Main Interface
def create_main_interface():
    """Create the main interface with tabs"""
    
    # Create tabs
    embedding_tab = create_embedding_interface()
    verification_tab = create_verification_interface()
    
    # Create tab container
    tab = widgets.Tab()
    tab.children = [embedding_tab, verification_tab]
    tab.set_title(0, '🔒 EMBED')
    tab.set_title(1, '🔍 VERIFY')
    
    # Display everything
    display(widgets.VBox([
        widgets.HTML("""
            <h1 style="text-align: center; color: #2E86AB;">🔒 Fragile Watermarking System</h1>
            <p style="text-align: center; color: #666;">Separate Embedding and Verification Workflow</p>
            <hr>
        """),
        tab
    ]))

In [7]:
# 6. SIMPLE FALLBACK INTERFACE
def create_simple_interface():
    """Simple interface as fallback"""
    
    print("🔧 Loading simple interface...")
    
    def simple_embed():
        from google.colab import files
        import io
        
        print("=== SIMPLE EMBEDDING ===")
        uploaded = files.upload()
        if not uploaded:
            print("No file uploaded!")
            return
            
        filename = list(uploaded.keys())[0]
        watermark = input("Enter watermark text: ").strip()
        if not watermark:
            watermark = "© Copyright 2024"
            
        # Process image
        image = Image.open(io.BytesIO(uploaded[filename]))
        img_array = np.array(image)
        mode = 'color' if len(img_array.shape) == 3 else 'grayscale'
        
        watermarked, wm_length = embed_watermark_lsb(img_array, watermark, mode)
        
        # Save and show
        Image.fromarray(watermarked).save('watermarked.png')
        files.download('watermarked.png')
        
        print(f"✅ Saved! Watermark length: {wm_length}")
        print(f"💡 Use this length for verification: {wm_length}")
    
    def simple_verify():
        from google.colab import files
        import io
        
        print("=== SIMPLE VERIFICATION ===")
        uploaded = files.upload()
        if not uploaded:
            print("No file uploaded!")
            return
            
        filename = list(uploaded.keys())[0]
        expected = input("Expected watermark: ").strip()
        wm_length = input("Watermark length (or press Enter to calculate): ").strip()
        
        if not wm_length:
            wm_length = len(text_to_binary(expected)) + 8
        else:
            wm_length = int(wm_length)
        
        # Process image
        image = Image.open(io.BytesIO(uploaded[filename]))
        img_array = np.array(image)
        mode = 'color' if len(img_array.shape) == 3 else 'grayscale'
        
        extracted = extract_watermark_lsb(img_array, wm_length, mode)
        print(f"Expected: {expected}")
        print(f"Extracted: {extracted}")
        print(f"Match: {expected in extracted}")
    
    # Offer simple interface
    print("Choose an option:")
    print("1. Simple Embedding")
    print("2. Simple Verification")
    choice = input("Enter 1 or 2: ")
    
    if choice == '1':
        simple_embed()
    elif choice == '2':
        simple_verify()
    else:
        print("Using advanced interface...")
        create_main_interface()

# 7. RUN THE SYSTEM
try:
    print("🚀 Initializing Fragile Watermarking System...")
    create_main_interface()
except Exception as e:
    print(f"⚠️  Advanced interface failed, using simple interface: {e}")
    create_simple_interface()

🚀 Initializing Fragile Watermarking System...
